**Definizione delle funzioni di identificazione dei bordi, identificazione dei bordi opposti e di crop+resize**

In [ ]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image

def load_image(file):
    img = Image.open(file)
    img = img.convert('L')
    img = np.array(img)
    return img

def save_image(img, file):
    img = Image.fromarray(img, mode='L')
    img.save(file)

def findBorders(t2w):
    # Make t2w boolean
    canale_bw = t2w > 0
    # Find non-black rows and columns
    rows = np.any(canale_bw, axis=1)
    cols = np.any(canale_bw, axis=0)
    # Get first and last non-black row and column indexes
    rmin, rmax = np.where(rows)[0][[0, -1]] if np.any(rows) else (0, t2w.shape[0])
    cmin, cmax = np.where(cols)[0][[0, -1]] if np.any(cols) else (0, t2w.shape[1])
    return rmin, rmax, cmin, cmax

def findBordersOpposite(adc, rmin, rmax, cmin, cmax):
    # Calculate opposite coordinates for cropping
    opp_rmin = adc.shape[0] - rmax
    opp_rmax = adc.shape[0] - rmin
    opp_cmin = adc.shape[1] - cmax
    opp_cmax = adc.shape[1] - cmin
    return opp_rmin, opp_rmax, opp_cmin, opp_cmax

def cropResizeImages(adc, hbv, t2w, mask):
    # Get original image size
    original_shape = t2w.shape
    
    # Find black borders in t2w channel
    rmin_t2w, rmax_t2w, cmin_t2w, cmax_t2w = findBorders(t2w)
    
    # Calculate opposite borders for adc and hbv
    rmin_adc, rmax_adc, cmin_adc, cmax_adc = findBordersOpposite(adc, rmin_t2w, rmax_t2w, cmin_t2w, cmax_t2w)
    
    # Crop t2w
    t2w_cropped = t2w[rmin_t2w:rmax_t2w, cmin_t2w:cmax_t2w]
    mask_cropped = mask[rmin_t2w:rmax_t2w, cmin_t2w:cmax_t2w]
    
    # Crop adc and hbv at opposite borders
    adc_cropped = adc[rmin_adc:rmax_adc, cmin_adc:cmax_adc]
    hbv_cropped = hbv[rmin_adc:rmax_adc, cmin_adc:cmax_adc]
    
    cropped_shape = t2w_cropped.shape
    
    # RESIZE IMAGES
    # Resize cropped images to the original shape
    resizeShape = t2w.shape
    adc_cropped_resized = np.array(Image.fromarray(adc_cropped, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    hbv_cropped_resized = np.array(Image.fromarray(hbv_cropped, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    t2w_cropped_resized = np.array(Image.fromarray(t2w_cropped, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    mask_cropped_resized = np.array(Image.fromarray(mask_cropped, mode='L').resize(resizeShape, Image.NEAREST), dtype=np.uint8)
    
    # Create a dictionary to store crop metadata
    # cropMetadata = {
    #     't2w_start_x': cmin_t2w, 't2w_start_y': rmin_t2w, 't2w_end_x': cmax_t2w, 't2w_end_y': rmax_t2w,
    #     'adc_start_x': cmin_adc, 'adc_start_y': rmin_adc, 'adc_end_x': cmax_adc, 'adc_end_y': rmax_adc,
    #     'cropDimension': min(rmax_t2w - rmin_t2w, cmax_t2w - cmin_t2w),
    #     'cropped_shape': cropped_shape,
    #     'original_shape': original_shape,
    #     'resizeShape': resizeShape,
    #     'flagCrop': True
    # }
    
    return adc_cropped_resized, hbv_cropped_resized, t2w_cropped_resized, mask_cropped_resized


**Applicazione del crop ai pazienti rientranti nella casistica 2**

In [ ]:
# Path in cui sono presenti le cartelle dei pazienti
pathDataset = os.path.join(".", "PAZIENTI CASISTICA 2")

# Loop sui pazienti della casistica 2
for paziente in tqdm(os.listdir(pathDataset)):
    pathPaziente = os.path.join(pathDataset, paziente)
    # Loop sulle slice di un paziente della casistica 2
    for slice in os.listdir(os.path.join(pathPaziente, "adc")):
        # Load delle immagini
        adc = load_image(os.path.join(pathPaziente, "adc", slice))
        hbv = load_image(os.path.join(pathPaziente, "hbv", slice))
        t2w = load_image(os.path.join(pathPaziente, "t2w", slice))
        mask = load_image(os.path.join(pathPaziente, "manual", slice))
        # Crop e resize delle immagini
        adc_cropped_resized, hbv_cropped_resized, t2w_cropped_resized, mask_cropped_resized = cropResizeImages(adc, hbv, t2w, mask)
        # Creazione dell'immagine stacked
        stacked = np.stack([adc_cropped_resized, hbv_cropped_resized, t2w_cropped_resized], axis=-1)
        # Salvataggio delle nuove immagini
        save_image(adc_cropped_resized, os.path.join(pathPaziente, "adc", slice))
        save_image(hbv_cropped_resized, os.path.join(pathPaziente, "hbv", slice))
        save_image(t2w_cropped_resized, os.path.join(pathPaziente, "t2w", slice))
        save_image(mask_cropped_resized, os.path.join(pathPaziente, "manual", slice))
        stacked = Image.fromarray(stacked, mode='RGB')
        stacked.save(os.path.join(pathPaziente, "stacked", slice))